## DH Parameter Solver

In [30]:
import numpy as np
import sympy as sp

In [31]:
c = lambda angle: round(np.cos(angle),2)
s = lambda angle: round(np.sin(angle),2)

In [32]:
def DH_matrix(theta,alpha,a,d):
    mat = []
    for i in range(len(theta)):
        l = sp.symbols(f'l_{i+1}')
        mat.append([round(np.deg2rad(theta[i]),2), np.round(np.deg2rad(alpha[i]),2), a[i],l])
    return np.array(mat)

In [33]:
T = lambda dh:np.array([
    [c(dh[0]), -s(dh[0])*c(dh[1]), s(dh[0])*s(dh[1]), dh[2]*c(dh[0])],
    [s(dh[0]), c(dh[0])*c(dh[1]), -c(dh[0])*s(dh[1]), dh[2]*s(dh[0])],
    [0, s(dh[1]), c(dh[1]), dh[3]],
    [0,0,0,1]
    ])

In [34]:

def endEffectorPosition(dh, T):
    ans = []
    I = np.eye(4)
    for i in range(len(dh)):
        I = I@T(dh[i,:])
        ans = I
    return ans

In [ ]:
def manipulator_jacobian(DH_table, joint_types=None):
    """Compute the Jacobian matrix for the manipulator."""
    num_joints = DH_table.shape[0]
    T_total = sp.eye(4)
    J = sp.zeros(6, num_joints)
    z = sp.Matrix([0, 0, 1])
    o = sp.Matrix([0, 0, 0])

    if joint_types is None:
        joint_types = ['R'] * num_joints  # Assume all revolute if not specified

    for i in range(num_joints):
        T = T(DH_table[i, 0], DH_table[i, 1], DH_table[i, 2], DH_table[i, 3])
        T_total = T_total * T
        z = T_total[:3, 2]
        o = T_total[:3, 3]

        if joint_types[i] == 'R':  # Revolute joint
            J[:3, i] = z.cross(endEffectorPosition(DH_table) - o)
            J[3:, i] = z
        elif joint_types[i] == 'P':  # Prismatic joint
            J[:3, i] = z
            J[3:, i] = sp.Matrix([0, 0, 0])

    return J

In [35]:
theta = [0,90,0]
alpha = [-90,90,0]
a = [0,0,0]
d = 3

In [37]:
a = DH_matrix(theta,alpha,a,d)